In [15]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import date,timedelta
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import seaborn as sns
import pandas_ta as ta
import numpy as np
import json
from tkinter import *
from tkinter import messagebox, ttk, END
from matplotlib import dates

In [16]:
today = date.today()

d1 = today.strftime('%Y-%m-%d')
end_date = d1
d2 = date.today() - timedelta(days=365)
d2 = d2.strftime('%Y-%m-%d')
start_date = d2

In [17]:
from curl_cffi import requests
session = requests.Session(impersonate="chrome")
ticker = input('Write ticker: ')
stock_data = yf.download(ticker, start = start_date, end = end_date, progress=False, auto_adjust=False, session=session)
if isinstance(stock_data.columns, pd.MultiIndex):
    stock_data.columns = stock_data.columns.droplevel(1)
stock_data

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2025-02-18,95539.546875,95539.546875,96695.375000,93388.835938,95773.812500,37325720482
2025-02-19,96635.609375,96635.609375,96855.593750,95011.968750,95532.531250,28990872862
2025-02-20,98333.937500,98333.937500,98767.195312,96442.671875,96632.679688,31668022771
2025-02-21,96125.546875,96125.546875,99497.968750,94852.960938,98340.671875,49608706470
2025-02-22,96577.757812,96577.757812,96950.156250,95765.343750,96134.203125,18353824477
...,...,...,...,...,...,...
2026-02-13,68857.843750,68857.843750,69382.835938,65835.781250,66213.375000,40820775886
2026-02-14,69767.625000,69767.625000,70481.164062,68706.617188,68856.984375,36012397645
2026-02-15,68788.187500,68788.187500,70939.289062,68052.546875,69764.953125,40191152750


In [18]:
#RSI, Garman, ATR, MACD
stock_data['RSI'] = ta.rsi(close = stock_data["Adj Close"], length=14)

stock_data['Garman'] = ((np.log(stock_data['High'])- np.log(stock_data['Low']))**2)/2-(2*np.log(2)-1)*((np.log(stock_data['Adj Close']) -np.log(stock_data['Open']))**2)

def compute_atr(stock_data):
    atr = ta.atr(high=stock_data['High'], low=stock_data['Low'], close=stock_data['Close'], length=14)
    return atr.sub(atr.mean()).div(atr.std())
stock_data['ATR'] = compute_atr(stock_data)

def compute_macd(stock_data, short_period=12, long_period=26, signal_period=9):
    # Compute MACD, Signal Line, and Histogram using pandas_ta
    macd = ta.macd(
        close=stock_data['Close'], 
        fast=short_period, 
        slow=long_period, 
        signal=signal_period
    )
    
    # Assign the results to new columns in the DataFrame
    stock_data['MACD'] = macd['MACD_12_26_9']  # Default naming format
    stock_data['Signal'] = macd['MACDs_12_26_9']
    stock_data['MACD_Histogram'] = macd['MACDh_12_26_9']
    
    return stock_data

# Apply the function to compute MACD
stock_data = compute_macd(stock_data)

today = date.today() 
two_weeks = today - timedelta(days=14)

#Last 14 days
technical_analysis = stock_data.iloc[-14:]
technical_analysis

Price,Adj Close,Close,High,Low,Open,Volume,RSI,Garman,ATR,MACD,Signal,MACD_Histogram
Date,,,,,,,,,,,,
2026-02-04,73019.703125,73019.703125,76864.656250,71779.929688,75640.093750,67215363944,22.800072,0.001862,0.456679,-3965.867483,-2335.810888,-1630.056596
2026-02-05,62702.097656,62702.097656,73161.554688,62353.535156,73016.375000,125509410908,15.733813,0.003817,1.172502,-5162.678197,-2901.184349,-2261.493848
2026-02-06,70555.390625,70555.390625,71681.304688,60074.203125,62704.453125,114674259489,32.804503,0.010227,1.915883,-5415.042956,-3403.956071,-2011.086886
2026-02-07,69281.968750,69281.968750,71611.148438,67364.445312,70553.796875,62347107663,31.683677,0.001741,1.881372,-5652.638254,-3853.692508,-1798.945746
2026-02-08,70264.726562,70264.726562,72206.906250,68852.898438,69283.726562,39721722619,33.570051,0.001055,1.761420,-5695.974251,-4222.148856,-1473.825395
2026-02-09,70120.781250,70120.781250,71369.968750,68291.031250,70243.328125,52081598792,33.424470,0.000971,1.622949,-5676.498385,-4513.018762,-1163.479623
2026-02-10,68793.960938,68793.960938,70464.265625,67913.093750,70137.390625,40593063077,32.045003,0.000535,1.442400,-5702.393291,-4750.893668,-951.499623
2026-02-11,66991.968750,66991.968750,69242.679688,65757.304688,68791.859375,49671946030,30.220781,0.001062,1.366739,-5801.445449,-4961.004024,-840.441425
2026-02-12,66221.843750,66221.843750,68339.492188,65092.109375,66992.195312,44651071271,29.449196,0.001133,1.273047,-5874.371550,-5143.677529,-730.694021


In [19]:
import plotly.graph_objects as go

# ----------------------------
# Indicators
# ----------------------------
stock_data["M10"] = stock_data["Close"].rolling(window=10).mean()
stock_data["M20"] = stock_data["Close"].rolling(window=20).mean()

stock_data["M40"] = stock_data["Close"].rolling(window=40).mean()
stock_data["M80"] = stock_data["Close"].rolling(window=80).mean()

stock_data["SD"] = stock_data["Close"].rolling(window=20).std()
stock_data["UB"] = stock_data["M20"] + 2 * stock_data["SD"]
stock_data["LB"] = stock_data["M20"] - 2 * stock_data["SD"]


fig = go.Figure()

# 0 Close (line)
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["Close"],
    mode="lines", name="Close"
))

# 1 Candles
fig.add_trace(go.Candlestick(
    x=stock_data.index,
    open=stock_data["Open"],
    high=stock_data["High"],
    low=stock_data["Low"],
    close=stock_data["Close"],
    name="Candles"
))

# 2-3 MA 10/20
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M10"], mode="lines", name="M10"))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M20"], mode="lines", name="M20"))

# 4-5 MA 40/80
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M40"], mode="lines", name="M40"))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["M80"], mode="lines", name="M80"))

# 6-8 Bollinger Bands
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["UB"],
    mode="lines", name="Upper BB"
))
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["LB"],
    mode="lines", name="Lower BB",
    fill="tonexty"  # fill between UB (previous trace) and LB
))
fig.add_trace(go.Scatter(
    x=stock_data.index, y=stock_data["M20"],
    mode="lines", name="Middle BB (M20)"
))

# ----------------------------
# Visibility presets (length MUST be 9)
# ----------------------------
PRICE_CANDLES = [False, True,  False, False, False, False, False, False, False]
PRICE_LINE    = [True,  False, False, False, False, False, False, False, False]

MA_10_20      = [True,  False, True,  True,  False, False, False, False, False]
MA_40_80      = [True,  False, False, False, True,  True,  False, False, False]

BB_VIEW       = [True,  False, False, True,  False, False, True,  True,  True]
# (BB_VIEW includes Close + M20 + UB/LB + Middle; you can tweak if you want)

# Default view
for t in fig.data:
    t.visible = False
fig.data[1].visible = True  # default = candles

# ----------------------------
# Layout + Buttons (positioned to avoid title conflict)
# ----------------------------
fig.update_layout(
    title=dict(text=f"{ticker} Price", x=0.01, y=0.97),  # keep title slightly lower
    xaxis_title="Date",
    yaxis_title="Price",
    plot_bgcolor="lightgrey",
    xaxis_rangeslider_visible=False,
    width=1050,
    height=650,
    margin=dict(t=110, r=30, l=60, b=50),  # extra top margin for buttons + title
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=1.0,
            y=1.16,
            xanchor="right",
            yanchor="top",
            pad={"r": 10, "t": 10},
            buttons=[
                dict(
                    label="Candles",
                    method="update",
                    args=[{"visible": PRICE_CANDLES},
                          {"title": f"{ticker} Price (Candles)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="Line",
                    method="update",
                    args=[{"visible": PRICE_LINE},
                          {"title": f"{ticker} Price (Line)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="MA 10/20",
                    method="update",
                    args=[{"visible": MA_10_20},
                          {"title": f"{ticker} Moving Averages (10/20)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="MA 40/80",
                    method="update",
                    args=[{"visible": MA_40_80},
                          {"title": f"{ticker} Moving Averages (40/80)", "yaxis": {"title": "Price"}}],
                ),
                dict(
                    label="Bollinger",
                    method="update",
                    args=[{"visible": BB_VIEW},
                          {"title": f"{ticker} Bollinger Bands (20, 2σ)", "yaxis": {"title": "Price"}}],
                ),
            ],
        )
    ],
)

fig.show()


Sharpe and Sortino Ratio

In [20]:
stock_data = stock_data.copy()
stock_data["Daily Returns"] = stock_data["Adj Close"].pct_change()
stock_data.dropna(inplace=True)

def compute_sharpe_ratio(data, risk_free_rate=0, periods=365):
    """
    Computes daily and annualized Sharpe Ratio
    """
    returns = data["Daily Returns"]
    excess_returns = returns - risk_free_rate / periods

    daily_sharpe = excess_returns.mean() / returns.std()
    annual_sharpe = daily_sharpe * np.sqrt(periods)

    return daily_sharpe, annual_sharpe

def compute_sortino_ratio(data, threshold=0, risk_free_rate=0, periods=365):
    """
    Computes annualized Sortino Ratio
    """
    returns = data["Daily Returns"]
    downside_returns = returns[returns < threshold]

    downside_std = downside_returns.std()
    excess_return = returns.mean() - risk_free_rate / periods

    sortino = (excess_return / downside_std) * np.sqrt(periods)
    return sortino


daily_sharpe, annual_sharpe = compute_sharpe_ratio(stock_data)
sortino = compute_sortino_ratio(stock_data)

print(f"Daily Sharpe Ratio for {ticker}: {daily_sharpe:.4f}")
print(f"Annual Sharpe Ratio for {ticker}: {annual_sharpe:.4f}")
print(f"Sortino Ratio for {ticker}: {sortino:.4f}")


Daily Sharpe Ratio for BTC-USD: -0.0457
Annual Sharpe Ratio for BTC-USD: -0.8736
Sortino Ratio for BTC-USD: -1.1360


In [21]:
prices = pd.concat([xrp, btc, eth], axis=1)
prices.columns = ["XRP", "BTC", "ETH"]
prices = prices.dropna()

# daily log returns
log_returns = np.log(prices / prices.shift(1)).dropna()

def monte_carlo_paths(price_series, mu, sigma, days=365, n_sims=5000, seed=42):
    """
    Geometric Brownian Motion (GBM) simulation using log returns
    """
    rng = np.random.default_rng(seed)
    s0 = float(price_series.iloc[-1])

    # random daily shocks
    z = rng.standard_normal((days, n_sims))

    # GBM in log space: S(t) = S0 * exp( (mu - 0.5*sigma^2)*t + sigma*W )
    drift = (mu - 0.5 * sigma**2)
    daily_log_steps = drift + sigma * z
    log_paths = np.cumsum(daily_log_steps, axis=0)
    paths = s0 * np.exp(log_paths)

    # add S0 as day 0
    paths = np.vstack([np.full((1, n_sims), s0), paths])
    return paths  # shape: (days+1, n_sims)

def summarize_paths(paths):
    terminal = paths[-1, :]
    s0 = paths[0, 0]
    summary = {
        "Start": s0,
        "Mean End": terminal.mean(),
        "Median End": np.median(terminal),
        "P05 End": np.percentile(terminal, 5),
        "P95 End": np.percentile(terminal, 95),
        "Prob End > Start": (terminal > s0).mean()
    }
    return summary, terminal

DAYS = 365
N_SIMS = 50000

results = {}
terminals = {}

for coin in ["XRP", "BTC", "ETH"]:
    mu = log_returns[coin].mean()
    sigma = log_returns[coin].std()

    paths = monte_carlo_paths(prices[coin], mu, sigma, days=DAYS, n_sims=N_SIMS, seed=42)
    summary, terminal = summarize_paths(paths)

    results[coin] = summary
    terminals[coin] = terminal

# Print results
for coin, s in results.items():
    print(f"\n--- {coin} Monte Carlo ({DAYS} days, {N_SIMS} sims) ---")
    print(f"Start:            {s['Start']:.6f}")
    print(f"Mean End:         {s['Mean End']:.6f}")
    print(f"Median End:       {s['Median End']:.6f}")
    print(f"5th % End:        {s['P05 End']:.6f}")
    print(f"95th % End:       {s['P95 End']:.6f}")
    print(f"Prob End > Start: {s['Prob End > Start']*100:.2f}%")


coin_list = ["XRP", "BTC", "ETH"]

PLOT_SIMS = 200
plot_paths = {}

for coin in coin_list:
    mu = log_returns[coin].mean()
    sigma = log_returns[coin].std()
    plot_paths[coin] = monte_carlo_paths(prices[coin], mu, sigma, days=DAYS, n_sims=PLOT_SIMS, seed=7)

fig = go.Figure()

trace_groups = {}

for i, coin in enumerate(coin_list):
    paths = plot_paths[coin]
    trace_idxs = []

    # spaghetti paths
    for s in range(paths.shape[1]):
        fig.add_trace(go.Scatter(
            x=np.arange(paths.shape[0]),
            y=paths[:, s],
            mode="lines",
            name=f"{coin} sim",
            line=dict(width=1),
            opacity=0.15,
            showlegend=False,
            visible=(i == 0)
        ))
        trace_idxs.append(len(fig.data) - 1)

    # mean path
    fig.add_trace(go.Scatter(
        x=np.arange(paths.shape[0]),
        y=paths.mean(axis=1),
        mode="lines",
        name=f"{coin} mean",
        line=dict(width=3),
        visible=(i == 0)
    ))
    trace_idxs.append(len(fig.data) - 1)

    trace_groups[coin] = trace_idxs

buttons = []
for coin in coin_list:
    visible = [False] * len(fig.data)
    for idx in trace_groups[coin]:
        visible[idx] = True

    buttons.append(dict(
        label=coin,
        method="update",
        args=[
            {"visible": visible},
            {"title": f"{coin} Monte Carlo Paths ({DAYS} days, {PLOT_SIMS} sims shown)",
             "xaxis": {"title": "Day"},
             "yaxis": {"title": "Simulated Price"}}
        ],
    ))

fig.update_layout(
    title=f"XRP Monte Carlo Paths ({DAYS} days, {PLOT_SIMS} sims shown)",
    width=1050,
    height=600,
    plot_bgcolor="lightgrey",
    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=1.0,
        y=1.15,
        xanchor="right",
        yanchor="top",
        pad={"r": 10, "t": 10},
        buttons=buttons
    )],
    margin=dict(t=110, r=30, l=60, b=50)
)

fig.show()

fig2 = go.Figure()

hist_groups = {}
for i, coin in enumerate(coin_list):
    terminal = terminals[coin]
    fig2.add_trace(go.Histogram(
        x=terminal,
        nbinsx=60,
        name=f"{coin} terminal",
        opacity=0.75,
        visible=(i == 0)
    ))
    hist_groups[coin] = len(fig2.data) - 1

buttons2 = []
for coin in coin_list:
    visible = [False] * len(fig2.data)
    visible[hist_groups[coin]] = True

    s = results[coin]
    subtitle = (f"Start={s['Start']:.6f} | Mean={s['Mean End']:.6f} | "
                f"P05={s['P05 End']:.6f} | P95={s['P95 End']:.6f} | "
                f"P(End>Start)={s['Prob End > Start']*100:.2f}%")

    buttons2.append(dict(
        label=coin,
        method="update",
        args=[
            {"visible": visible},
            {"title": f"{coin} Terminal Price Distribution<br><sup>{subtitle}</sup>",
             "xaxis": {"title": "Terminal Price"},
             "yaxis": {"title": "Count"}}
        ],
    ))

fig2.update_layout(
    title="XRP Terminal Price Distribution",
    width=1050,
    height=600,
    plot_bgcolor="lightgrey",
    updatemenus=[dict(
        type="buttons",
        direction="right",
        x=1.0,
        y=1.15,
        xanchor="right",
        yanchor="top",
        pad={"r": 10, "t": 10},
        buttons=buttons2
    )],
    margin=dict(t=110, r=30, l=60, b=50)
)

fig2.show()


NameError: name 'xrp' is not defined